In [1]:
if not('init' in globals()):
    import os
    os.chdir('../..')

    print("Current working directory: {0}".format(os.getcwd()))

    import torch 
    import torch.nn as nn

    from tests.utils import *

    import model.layer
    import model.quantizer

    init = 0


Current working directory: /home/dschnoell/sources/sim2


In [5]:
shape = (1,30,1,1)

add = model.layer.AddQAT(size=shape,out_quant_bits=8,out_quant_channel_wise=True,)

a_quant = model.quantizer.LinQuantExpScale(8,shape,1)
b_quant = model.quantizer.LinQuantExpScale(8,shape,1)


In [70]:
size=(100,shape[1],20,30)

a = torch.randn(size)
b = torch.randn(size)

a_quant.train()
b_quant.train()
add.train()

aq = (a_quant(a.clone()),a_quant.delta_out.log2())
bq = (b_quant(b.clone()),b_quant.delta_out.log2())

pr("diff of indata a",a,aq[0]) 
print(aq[1].exp2().view(-1))
pr("diff of indata b",b,bq[0])
print(bq[1].exp2().view(-1))

pr("diff of indata a",a,aq[0].div((aq[1]).exp2()).floor().mul((aq[1]).exp2())) 


c = add(aq,bq)
# print(c)

pr("diff in train to float",c[0],a+b)
pr("diff in train to quant",c[0],aq[0]+bq[0])
print(c[1].exp2().view(-1))
print(c[1].shape)


a_quant.eval()
b_quant.eval()
add.eval()

aq2 = (a_quant(a.clone()),a_quant.delta_out.log2())
bq2 = (b_quant(b.clone()),b_quant.delta_out.log2())


pr("diff of a",aq[0]/(aq2[1].exp2()),aq2[0])

# print(aq[0]/)

c2 = add(aq2,bq2)

pr("diff train eval",c[0]/(c2[1].exp2()),c2[0])
# print(c[0]/(c2[1].exp2())-c2[0])
print(c2[1].exp2().view(-1))
print(c2[1].shape)
print(add.a_shift.view(-1))
print(add.b_shift.view(-1))


diff of indata a diff: 0.0627450942993164, mean: 0.03137027099728584
tensor([0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627,
        0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627,
        0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627,
        0.0627, 0.0627, 0.0627])
diff of indata b diff: 0.06274503469467163, mean: 0.03136405721306801
tensor([0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627,
        0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627,
        0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627, 0.0627,
        0.0627, 0.0627, 0.0627])
diff of indata a diff: 0.0627450942993164, mean: 0.03137027099728584
Hello from high pres
diff in train to float diff: 0.25085973739624023, mean: 0.12548013031482697
diff in train to quant diff: 0.12549066543579102, mean: 0.06274580210447311
tensor([0.1255, 0.1255, 0.1255, 0.1255, 0.1255, 0.1255, 0.1255, 0.1255, 0